https://projecteuler.net/problem=secret

In [174]:
import cv2
import numpy as np
from tqdm import tqdm
import numpy as np
from functools import cache

In [ ]:
import numpy as np

def C(A, B, mod=7):
    n, m = A.shape
    p, q = B.shape
    C = np.zeros((n, m), dtype=int)

    for i in range(n):
        for j in range(m):
            s = 0
            for x in range(p):
                for y in range(q):
                    ii = (i + x - p//2) % n
                    jj = (j + y - q//2) % m
                    s += A[ii, jj] * B[x, y]
            C[i, j] = s % mod
    return C

def C_p(A, p, mod=7):
    n, m = A.shape
    result = np.zeros((n, m), dtype=int)
    result[n//2, m//2] = 1  
    base = A.copy()

    while p > 0:
        if p % 2 == 1:
            result = C(result, base, mod)
        base = C(base, base, mod)
        p //= 2

    return result


In [231]:
def arr_to_img(arr):
    arr = (arr * (255 // 6)).astype(np.uint8)
    cv2.imwrite("output.jpg", arr)

In [235]:
def fast_convolution_power(I, K, p, mod=7):
    n, m = I.shape

    # Compute FFT of image and kernel
    F_I = np.fft.fft2(I)
    F_K = np.fft.fft2(K, s=(n, m))  # pad kernel to image size

    # Raise kernel spectrum to power p
    F_K_pow = np.power(F_K, p)

    # Apply convolution power in frequency domain
    F_result = F_I * F_K_pow
    result = np.fft.ifft2(F_result).real.round().astype(np.int64)

    # Apply modulo
    result = result % mod
    return result

In [236]:
I = np.array(cv2.imread("secret.jpg", cv2.IMREAD_GRAYSCALE).astype(np.int32))
K = np.array([[0,1,0], [1,0,1], [0,1,0]])
arr_to_img(fast_convolution_power(I, K, 10**12))

C:\Users\jadamek\AppData\Local\Temp\ipykernel_12996\1022398517.py:9: RuntimeWarning: overflow encountered in power
  F_K_pow = np.power(F_K, p)
C:\Users\jadamek\AppData\Local\Temp\ipykernel_12996\1022398517.py:12: RuntimeWarning: invalid value encountered in multiply
  F_result = F_I * F_K_pow
C:\Users\jadamek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\fft\_pocketfft.py:94: RuntimeWarning: invalid value encountered in ifft
  return ufunc(a, fct, axes=[(axis,), (), (axis,)], out=out)
C:\Users\jadamek\AppData\Local\Temp\ipykernel_12996\1022398517.py:13: RuntimeWarning: invalid value encountered in cast
  result = np.fft.ifft2(F_result).real.round().astype(np.int64)


In [237]:
fast_convolution_power(I, K, 10**12)

C:\Users\jadamek\AppData\Local\Temp\ipykernel_12996\1022398517.py:9: RuntimeWarning: overflow encountered in power
  F_K_pow = np.power(F_K, p)
C:\Users\jadamek\AppData\Local\Temp\ipykernel_12996\1022398517.py:12: RuntimeWarning: invalid value encountered in multiply
  F_result = F_I * F_K_pow
C:\Users\jadamek\AppData\Local\Temp\ipykernel_12996\1022398517.py:13: RuntimeWarning: invalid value encountered in cast
  result = np.fft.ifft2(F_result).real.round().astype(np.int64)


array([[6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       ...,
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6]], shape=(585, 975))